In [ ]:
import os
import requests
import logging
from tqdm.notebook import tqdm

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        logging.info(f"Created directory: {directory}")
    else:
        logging.info(f"Directory already exists: {directory}")

def download_file(url, dest_folder, chunk_size=1024*1024, verify_ssl=True):
    logging.info(f"Starting download of {url}")
    try:
        create_directory(dest_folder)
        local_filename = os.path.join(dest_folder, url.split('/')[-1])
        response = requests.get(url, stream=True, verify=verify_ssl)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        t = tqdm(total=total_size, unit='iB', unit_scale=True, desc=local_filename)
        
        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=chunk_size):
                t.update(len(chunk))
                f.write(chunk)
        t.close()
        
        if total_size != 0 and t.n != total_size:
            logging.error(f"Error downloading {local_filename}.")
        else:
            logging.info(f"Downloaded {local_filename}")
    except Exception as e:
        logging.error(f"Failed to download {url}. Error: {e}")

# URLs for various datasets
dataset_urls = {
    "owid": [
        "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/full_data.csv"
    ],
    "google_cloud": [
        "https://storage.googleapis.com/covid19-open-data/v2/main.csv"
    ],
    "csse": [
        "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv",
        "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
    ],
    "who": [
        "https://covid19.who.int/WHO-COVID-19-global-data.csv"
    ]
}

logging.info("Dataset URLs defined.")

# Download datasets from various sources
logging.info("Starting download of datasets from various sources.")
for source, urls in dataset_urls.items():
    for url in urls:
        # Disable SSL verification for specific URLs
        if "storage.googleapis.com" in url:
            download_file(url, f"datasets/{source}", verify_ssl=False)
        else:
            download_file(url, f"datasets/{source}")
logging.info("Finished downloading datasets from various sources.")
